In [1]:
import os 

In [10]:
import pandas as pd

df=pd.read_csv("winequality-red.csv")
df.head(2)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.0,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.0,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5


create entity or a data class to get the path variable from the config.yaml

In [11]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataValidationConfig:
    root_dir:Path
    unzip_data_dir:Path
    status_file:Path
    all_schema:dict

creating configuration manager

In [44]:
from mlproject.utils.common import read_yaml_file,create_directories
from mlproject.constant import *

class ConfigurationManager:
    def __init__(self,
                config_filepath=config_file_path,
                params_filepath=params_file_path,
                schema_filepath=schema_file_path):
        
        self.config=read_yaml_file(config_filepath)
        self.params=read_yaml_file(params_filepath)
        self.schema=read_yaml_file(schema_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_data_validation_config(self) -> DataValidationConfig:
        config=self.config.data_validation
        schema=self.schema.columns
        
        
        create_directories([config.root_dir])
        
        data_validation_config=DataValidationConfig(
            root_dir=config.root_dir,
            unzip_data_dir=config.unzip_data_dir,
            status_file=config.STATUS_FILE,
            all_schema=schema
        )
        
        return data_validation_config

now we'll create our components. it will take data validation columns

In [45]:
class DataValidation:
    def __init__(self,config=DataValidationConfig):
        self.config=config
        
    def validate_all_columns(self) -> bool:
        try:
            
            validation_status=None
            
            data=pd.read_csv(self.config.unzip_data_dir)
            all_cols=list(data.columns)
            
            all_schema=self.config.all_schema.keys()
            
            for col in all_cols:
                if col not in all_schema:
                    validation_status=False
                    with open(self.config.status_file,'w') as f:
                        f.write(f"validation status: {validation_status}")
                else:
                        validation_status=True
                        with open(self.config.status_file,'w') as f:
                            f.write(f"validation status: {validation_status}")
                            
            return validation_status
        except Exception as e:
            raise e

defining pipeline using config manager

In [47]:
try:
    config=ConfigurationManager()
    data_validation_config=config.get_data_validation_config()
    data_validation=DataValidation(config=data_validation_config)
    data_validation.validate_all_columns()
except Exception as e:
    raise e

[2024-08-12 15:35:18,816: INFO: common: yaml file: config\config.yaml loaded successfully:]
[2024-08-12 15:35:18,816: INFO: common: yaml file: params.yaml loaded successfully:]
[2024-08-12 15:35:18,816: INFO: common: yaml file: schema.yaml loaded successfully:]
[2024-08-12 15:35:18,816: INFO: common: created directory at artifacts:]
[2024-08-12 15:35:18,816: INFO: common: created directory at artifacts/data_validation:]
